In [9]:
import json
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import re #regukar expression
import warnings
import pytz #date-time conversion
import requests #to get info from server
import time
import datetime as dt
import json

from pandas.io.json import json_normalize
from IPython.display import display
from datetime import datetime
from pandas.io.json import json_normalize
from datetime import date, timedelta
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  #ignores the warning message and code runs faster # default='warn'

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
pd.options.display.float_format = '{:.2f}'.format

from IPython.display import display
from IPython.display import HTML
import geopy.distance
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#  Dark-sky : https://darksky.net/dev/docs

In [5]:
def getdarkskydatadaily(start,end,loc_name):
    """
    
    Input - start_date, end_date, API KEY from https://darksky.net/dev/register (now closed APPLE has acqired darksky)
    Output - daily level weather data from start date to end date for specified location
    
    """

    api_key ='03f7d7221972645470c7e8c6f35edd85'
    headers = {'User-Agent': 'Chrome/78and.0.3865.90'}
    http_proxy  = "http://10.245.5.249:8080"
    https_proxy = "https://10.245.5.249:8080"
    ftp_proxy   = "ftp://10.245.5.249:8080"

    proxyDict = { 
                "http"  : http_proxy, 
                "https" : https_proxy, 
                "ftp"   : ftp_proxy
                 }
    
    weather_data_daily = pd.DataFrame()
    date_list = []
    
    # get month,year and day from start date
    start_date = dt.datetime.strptime(start, "%Y-%m-%d")
    start_month = start_date.month
    start_year = start_date.year
    start_day = start_date.day
    
    # get month,year and day from end date
    end_date = dt.datetime.strptime(end, "%Y-%m-%d")
    end_month = end_date.month
    end_year = end_date.year
    end_day = end_date.day
    
    # store all date from start date to end date in date list 
    start_date = date(start_year, start_month, start_day)
    end_date = date(end_year, end_month, end_day)
    delta = timedelta(days=1)
    
    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += delta
    
    # latitude of the location markers
    sites_latitude = {
        'Marker 1' : '39.9613','Marker 2' : '39.8971','Marker 3' : '39.9060','Marker 4' : '39.9024','Marker 5' : '39.8960','Marker 6' : '39.8339',
        'Marker 7' : '39.8412','Marker 8' : '39.8381','Marker 9' : '39.8386','Marker 10' : '39.7579','Marker 11' : '39.7621','Marker 12' : '39.7621',
        'Marker 13' : '39.7695','Marker 14' : '39.6617','Marker 15' : '39.6639','Marker 16' : '39.6702','Marker 17' : '39.6744','Marker 18' : '39.5909',
        'Marker 19' : '39.5295','Marker 20' : '39.5475'
    }
    
    # longitude of the location markers
    sites_longitude = {
        'Marker 1' : '-86.4034','Marker 2' : '-86.3045','Marker 3' : '-86.2001','Marker 4' : '-86.0738','Marker 5' : '-85.9783','Marker 6' : '-86.3155',
        'Marker 7' : '-86.2056','Marker 8' : '-86.0985','Marker 9' : '-85.9811','Marker 10' : '-86.3155','Marker 11' : '-86.2042','Marker 12' : '-86.0923',
        'Marker 13' : '-85.9708','Marker 14' : '-86.2935','Marker 15' : '-86.1823','Marker 16' : '-86.0669','Marker 17' : '-85.9557','Marker 18' : '-86.4212',
        'Marker 19' : '-86.5874','Marker 20' : '-86.2743'
    }
    
    value1 = sites_latitude.get(loc_name)
    value2 = sites_longitude.get(loc_name)
    if(value1 == None and value2 == None):
        print('Unknown Location Name Used: %s',loc_name)
        print('\n')
        print('Use some other Consumer Location or Generation Location Name.. .. ..')
        print('\n')
    else :
        for i in range(len(date_list)):
            #requesting web page
            date_format = '%Y-%m-%dT%H:%M:%S'
            dates = dt.datetime.strptime(date_list[i], '%Y-%m-%d').strftime(date_format)
            links = 'https://api.darksky.net/forecast/'+api_key+'/'+sites_latitude.get(loc_name)+','+sites_longitude.get(loc_name)+','+str(dates)+'?exclude=currently,hourly,alerts?units=us'
            print(links)
            response = requests.get(links,headers=headers,proxies=proxyDict)

            # converting into json
            weather = json.loads(response.content.decode('utf-8'))
            # getting it into data frame
            weather_data = json_normalize(weather['daily']['data'])
            weather_data_daily = weather_data_daily.append(weather_data,sort=True)
    
    weather_data_daily.reset_index(drop=True, inplace=True)
    
    return weather_data_daily

In [7]:
start_date = "2020-04-06"
end_date = "2020-04-07"

sites_latitude = {
    'Marker 1' : '39.9613','Marker 2' : '39.8971','Marker 3' : '39.9060','Marker 4' : '39.9024','Marker 5' : '39.8960','Marker 6' : '39.8339',
    'Marker 7' : '39.8412','Marker 8' : '39.8381','Marker 9' : '39.8386','Marker 10' : '39.7579','Marker 11' : '39.7621','Marker 12' : '39.7621',
    'Marker 13' : '39.7695','Marker 14' : '39.6617','Marker 15' : '39.6639','Marker 16' : '39.6702','Marker 17' : '39.6744','Marker 18' : '39.5909',
    'Marker 19' : '39.5295','Marker 20' : '39.5475'
    }
    
# longitude of the location markers
sites_longitude = {
    'Marker 1' : '-86.4034','Marker 2' : '-86.3045','Marker 3' : '-86.2001','Marker 4' : '-86.0738','Marker 5' : '-85.9783','Marker 6' : '-86.3155',
    'Marker 7' : '-86.2056','Marker 8' : '-86.0985','Marker 9' : '-85.9811','Marker 10' : '-86.3155','Marker 11' : '-86.2042','Marker 12' : '-86.0923',
    'Marker 13' : '-85.9708','Marker 14' : '-86.2935','Marker 15' : '-86.1823','Marker 16' : '-86.0669','Marker 17' : '-85.9557','Marker 18' : '-86.4212',
    'Marker 19' : '-86.5874','Marker 20' : '-86.2743'
    }


location_marker = ['Marker 1', 'Marker 2', 'Marker 3', 'Marker 4', 'Marker 5', 'Marker 6', 'Marker 7', 'Marker 8', 'Marker 9', 'Marker 10',
                   'Marker 11', 'Marker 12', 'Marker 13', 'Marker 14', 'Marker 15', 'Marker 16', 'Marker 17', 'Marker 18', 'Marker 19', 'Marker 20']

darkskyfiles = []
weathersource = []
for i in location_marker:
    darksky_data = getdarkskydatadaily(start_date,end_date,i)
    darksky_data['Location'] = i
    darkskyfiles.append(darksky_data)

darksky_df = pd.concat(darkskyfiles)

darksky_df.reset_index(drop=True, inplace=True)

darksky_df.to_csv('darksky6th-10thApril2020')

https://api.darksky.net/forecast/03f7d7221972645470c7e8c6f35edd85/39.9613,-86.4034,2020-04-06T00:00:00?exclude=currently,hourly,alerts?units=us
https://api.darksky.net/forecast/03f7d7221972645470c7e8c6f35edd85/39.9613,-86.4034,2020-04-07T00:00:00?exclude=currently,hourly,alerts?units=us
https://api.darksky.net/forecast/03f7d7221972645470c7e8c6f35edd85/39.8971,-86.3045,2020-04-06T00:00:00?exclude=currently,hourly,alerts?units=us
https://api.darksky.net/forecast/03f7d7221972645470c7e8c6f35edd85/39.8971,-86.3045,2020-04-07T00:00:00?exclude=currently,hourly,alerts?units=us
https://api.darksky.net/forecast/03f7d7221972645470c7e8c6f35edd85/39.9060,-86.2001,2020-04-06T00:00:00?exclude=currently,hourly,alerts?units=us
https://api.darksky.net/forecast/03f7d7221972645470c7e8c6f35edd85/39.9060,-86.2001,2020-04-07T00:00:00?exclude=currently,hourly,alerts?units=us
https://api.darksky.net/forecast/03f7d7221972645470c7e8c6f35edd85/39.9024,-86.0738,2020-04-06T00:00:00?exclude=currently,hourly,alerts?u

In [8]:
darksky_df.head()

,apparentTemperatureHigh,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureLowTime,apparentTemperatureMax,apparentTemperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,cloudCover,dewPoint,humidity,icon,moonPhase,ozone,precipIntensity,precipIntensityMax,precipIntensityMaxTime,precipProbability,precipType,pressure,summary,sunriseTime,sunsetTime,temperatureHigh,temperatureHighTime,temperatureLow,temperatureLowTime,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,time,uvIndex,uvIndexTime,visibility,windBearing,windGust,windGustTime,windSpeed,Location
0,65.02,1586210400,54.25,1586256780,65.02,1586210400,36.89,1586170980,0.37,45.20,0.77,partly-cloudy-day,0.46,345.10,0.00,0.00,1586168520,0.10,rain,1018.70,Partly cloudy throughout the day.,1586172120,1586218560,65.52,1586210400,53.76,1586256780,65.52,1586210400,38.82,1586170980,1586145600,5,1586193900,9.86,178,13.77,1586207280,4.64,Marker 1
1,76.81,1586289780,59.91,1586345520,76.81,1586289780,54.25,1586256780,0.34,56.22,0.73,partly-cloudy-day,0.50,339.30,0.00,0.00,1586264400,0.15,rain,1009.60,Partly cloudy throughout the day.,1586258400,1586305020,77.31,1586289780,59.42,1586345520,77.31,1586289780,53.76,1586256780,1586232000,5,1586279460,10.00,223,28.31,1586314800,10.27,Marker 1
2,65.96,1586210460,54.78,1586256900,65.96,1586210460,37.98,1586171520,0.39,45.57,0.77,partly-cloudy-day,0.46,344.80,0.00,0.00,1586168460,0.08,rain,1018.70,Partly cloudy throughout the day.,1586172060,1586218560,66.46,1586210460,54.29,1586256900,66.46,1586210460,39.40,1586171400,1586145600,5,1586193840,9.92,180,12.53,1586206740,3.91,Marker 2
3,77.56,1586289780,61.08,1586347200,77.56,1586289780,54.78,1586256900,0.35,56.47,0.72,partly-cloudy-day,0.50,339.10,0.00,0.00,1586264400,0.14,rain,1009.70,Partly cloudy throughout the day.,1586258400,1586305020,78.06,1586289780,60.59,1586347200,78.06,1586289780,54.29,1586256900,1586232000,5,1586279760,10.00,222,25.80,1586314800,8.70,Marker 2
4,65.72,1586210580,54.33,1586256900,65.72,1586210580,37.84,1586171580,0.38,45.44,0.77,partly-cloudy-day,0.46,345.00,0.00,0.00,1586168340,0.08,rain,1018.80,Partly cloudy throughout the day.,1586172060,1586218500,66.22,1586210580,53.84,1586256900,66.22,1586210580,39.28,1586171400,1586145600,5,1586194200,9.96,178,12.46,1586206800,3.81,Marker 3
